In [1]:
import cv2
import numpy as np
import os

In [2]:
def get_line_pixels(image, x0, y0, x1, y1):
    # Implement the Bresenham's line algorithm
    points = []
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    x, y = x0, y0
    sx = -1 if x0 > x1 else 1
    sy = -1 if y0 > y1 else 1
    is_horizontal = dx > dy

    if is_horizontal:
        err = dx / 2.0
        while x != x1:
            if 0 <= x < len(image[0]) and 0 <= y < len(image):
                if 0 <= y-1 and y+1 < len(image):
                    points.append((x, y))
            err -= dy
            if err < 0:
                y += sy
                err += dx
            x += sx
    else:
        err = dy / 2.0
        while y != y1:
            if 0 <= x < len(image[0]) and 0 <= y < len(image):
                if 0 <= x-1 and x+1 < len(image[0]):
                    points.append((x, y))
            err -= dx
            if err < 0:
                x += sx
                err += dy
            y += sy
    if 0 <= x < len(image[0]) and 0 <= y < len(image):
        if (not is_horizontal) and 0 <= x-1 and x+1 < len(image[0]):
            points.append((x, y))
        if is_horizontal and 0 <= y-1 and y+1 < len(image):
            points.append((x, y))

    # Get the pixel values
    if is_horizontal:
        pixels = [(image[y-1][x], image[y+1][x]) for x, y in points]
    else:
        pixels = [(image[y][x-1], image[y][x+1]) for x, y in points]

    return pixels

In [3]:
def get_best_side_pair(image, is_horizontal, max_angle_dev, max_d_from_image_side, min_line_length):
    im_size_x = len(image[0])
    im_size_y = len(image)
    if is_horizontal:
        im_size = im_size_y
        min_angle_d = -max_angle_dev
        max_angle_d = max_angle_dev
        mid_angle_d = 0
    else:
        im_size = im_size_x
        min_angle_d = 90 - max_angle_dev
        max_angle_d = 90 + max_angle_dev
        mid_angle_d = 90

    all_values = []
    for angle_d in range(min_angle_d, max_angle_d):
        angle = np.radians(angle_d)
        if is_horizontal:
            line_length = int(im_size_x / np.cos(angle))
        else:
            line_length = int(im_size_y / np.sin(angle))

        all_contrasts = []
        for i in range(max_d_from_image_side):
            if is_horizontal:
                if mid_angle_d > angle_d:
                    x0, y0 = 0, i
                    x1, y1 = int(x0 + line_length * np.cos(angle)), int(y0 + line_length * np.sin(angle))
                else:
                    x1, y1 = im_size_x, i
                    x0, y0 = int(x1 - line_length * np.cos(angle)), int(y1 - line_length * np.sin(angle))
            else:
                if mid_angle_d > angle_d:
                    x1, y1 = i, im_size_y
                    x0, y0 = int(x1 - line_length * np.cos(angle)), int(y1 - line_length * np.sin(angle))
                else:
                    x0, y0 = i, 0
                    x1, y1 = int(x0 + line_length * np.cos(angle)), int(y0 + line_length * np.sin(angle))

            pixels = get_line_pixels(image, x0, y0, x1, y1)
            pixels = np.array(pixels, dtype=np.int16)
            if pixels.shape[0] < min_line_length:
                continue
            total_contrast = pixels[:, 0] - pixels[:, 1]
            all_contrasts.append((i, total_contrast.mean()))
        
        if len(all_contrasts) == 0:
            continue
        all_contrasts = np.array(all_contrasts)
        min_index = all_contrasts[all_contrasts[:, 1].argmin()][0]
        min_value = all_contrasts[:, 1].min()

        all_contrasts = []
        for i in range(im_size - max_d_from_image_side, im_size):
            if is_horizontal:
                if mid_angle_d > angle_d:
                    x1, y1 = im_size_x, i
                    x0, y0 = int(x1 - line_length * np.cos(angle)), int(y1 - line_length * np.sin(angle))
                else:
                    x0, y0 = 0, i
                    x1, y1 = int(x0 + line_length * np.cos(angle)), int(y0 + line_length * np.sin(angle))
            else:
                if mid_angle_d > angle_d:
                    x0, y0 = i, 0
                    x1, y1 = int(x0 + line_length * np.cos(angle)), int(y0 + line_length * np.sin(angle))
                else:
                    x1, y1 = i, im_size_y
                    x0, y0 = int(x1 - line_length * np.cos(angle)), int(y1 - line_length * np.sin(angle))

            pixels = get_line_pixels(image, x0, y0, x1, y1)
            pixels = np.array(pixels, dtype=np.int16)
            if pixels.shape[0] < min_line_length:
                continue
            total_contrast = pixels[:, 0] - pixels[:, 1]
            all_contrasts.append((i, total_contrast.mean()))
        
        if len(all_contrasts) == 0:
            continue
        all_contrasts = np.array(all_contrasts)
        max_index = all_contrasts[all_contrasts[:, 1].argmax()][0]
        max_value = all_contrasts[:, 1].max()

        if is_horizontal:
            if mid_angle_d > angle_d:
                min_x0, min_y0 = 0, min_index
                min_x1, min_y1 = int(min_x0 + line_length * np.cos(angle)), int(min_y0 + line_length * np.sin(angle))
                max_x1, max_y1 = im_size_x, max_index
                max_x0, max_y0 = int(max_x1 - line_length * np.cos(angle)), int(max_y1 - line_length * np.sin(angle))
            else:
                min_x1, min_y1 = im_size_x, min_index
                min_x0, min_y0 = int(min_x1 - line_length * np.cos(angle)), int(min_y1 - line_length * np.sin(angle))
                max_x0, max_y0 = 0, max_index
                max_x1, max_y1 = int(max_x0 + line_length * np.cos(angle)), int(max_y0 + line_length * np.sin(angle))
        else:
            if mid_angle_d > angle_d:
                min_x1, min_y1 = min_index, im_size_y
                min_x0, min_y0 = int(min_x1 - line_length * np.cos(angle)), int(min_y1 - line_length * np.sin(angle))
                max_x0, max_y0 = max_index, 0
                max_x1, max_y1 = int(max_x0 + line_length * np.cos(angle)), int(max_y0 + line_length * np.sin(angle))
            else:
                min_x0, min_y0 = min_index, 0
                min_x1, min_y1 = int(min_x0 + line_length * np.cos(angle)), int(min_y0 + line_length * np.sin(angle))
                max_x1, max_y1 = max_index, im_size_y
                max_x0, max_y0 = int(max_x1 - line_length * np.cos(angle)), int(max_y1 - line_length * np.sin(angle))
        all_values.append((min_x0, min_y0, min_x1, min_y1, max_x0, max_y0, max_x1, max_y1, max_value-min_value, min_index, angle_d))
    
    if len(all_values) == 0:
        return None
    all_values = np.array(all_values)
    best_index = all_values[:, 8].argmax()
    return int(all_values[best_index, 0]), int(all_values[best_index, 1]), int(all_values[best_index, 2]), int(all_values[best_index, 3]), int(all_values[best_index, 4]), int(all_values[best_index, 5]), int(all_values[best_index, 6]), int(all_values[best_index, 7])
    

In [4]:
# Load the image
image = cv2.imread('cropped/AA AT-096-[0].jpg', 0)  # 0 loads the image in grayscale


In [5]:
def get_bounding_lines(image, max_angle_dev, min_line_length_percentage = 40, max_d_from_image_side_percentage = 45):
    im_size_x = len(image[0])
    im_size_y = len(image)
    min_line_length_x = int(min_line_length_percentage * im_size_x / 100)
    min_line_length_y = int(min_line_length_percentage * im_size_y / 100)

    max_d_from_image_side_x = int(max_d_from_image_side_percentage * im_size_x / 100)
    max_d_from_image_side_y = int(max_d_from_image_side_percentage * im_size_y / 100)

    # Get the best horizontal pair
    best_horizontal_pair = get_best_side_pair(image, True, max_angle_dev, max_d_from_image_side_y, min_line_length_x)
    # Get the best vertical pair
    best_vertical_pair = get_best_side_pair(image, False, max_angle_dev, max_d_from_image_side_x, min_line_length_y)

    return best_horizontal_pair, best_vertical_pair

In [6]:
def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return (x, y)

In [7]:
def transform_image(image, points):
    # Convert the input points to a NumPy array
    points = np.array(points, dtype=np.float32)

    # Calculate the bounding box for the source region
    min_x, min_y = np.min(points, axis=0)
    max_x, max_y = np.max(points, axis=0)

    # Calculate the aspect ratio of the source region
    aspect_ratio = (max_x - min_x) / (max_y - min_y)

    # Define the target rectangle based on the aspect ratio
    target_height = 300  # Adjust this value based on your preference
    target_width = int(target_height * aspect_ratio)

    target_rect = np.array([[0, 0], [target_width, 0], [target_width, target_height], [0, target_height]], dtype=np.float32)

    # Calculate the perspective transformation matrix
    matrix = cv2.getPerspectiveTransform(points, target_rect)

    # Apply the perspective transformation
    result = cv2.warpPerspective(image, matrix, (target_width, target_height))

    return result

In [8]:
# Get the bounding lines and show them on the image. Do this for all images in the folder
for filename in os.listdir('cropped'):
    image = cv2.imread('cropped/' + filename, 0)
    h, v = get_bounding_lines(image, 15, 70, 50)
    
    if h is None or v is None:
        print(filename + '  :  failed to find bounding lines')
        continue

    # Show the lines on the image
    image1 = image.copy()
    cv2.line(image1, (h[0], h[1]), (h[2], h[3]), 255, 2)
    cv2.line(image1, (h[4], h[5]), (h[6], h[7]), 255, 2)
    cv2.line(image1, (v[0], v[1]), (v[2], v[3]), 255, 2)
    cv2.line(image1, (v[4], v[5]), (v[6], v[7]), 255, 2)
    
    # Get the intersection points
    p1 = line_intersection(((h[0], h[1]), (h[2], h[3])), ((v[0], v[1]), (v[2], v[3])))
    p2 = line_intersection(((h[0], h[1]), (h[2], h[3])), ((v[4], v[5]), (v[6], v[7])))
    p3 = line_intersection(((h[4], h[5]), (h[6], h[7])), ((v[4], v[5]), (v[6], v[7])))
    p4 = line_intersection(((h[4], h[5]), (h[6], h[7])), ((v[0], v[1]), (v[2], v[3])))

    image2 = image.copy()
    # Show the points on the image
    cv2.circle(image2, (int(p1[0]), int(p1[1])), 5, 255, -1)
    cv2.circle(image2, (int(p2[0]), int(p2[1])), 5, 255, -1)
    cv2.circle(image2, (int(p3[0]), int(p3[1])), 5, 255, -1)
    cv2.circle(image2, (int(p4[0]), int(p4[1])), 5, 255, -1)

    # Get the rectangle image
    image3 = transform_image(image, [p1, p2, p3, p4])
    
    # Show all three images in one line as the output of the cell
    cv2.imshow('Bounding lines', image1)
    cv2.imshow('Bounding points', image2)
    cv2.imshow('Transformed image', image3)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



KeyboardInterrupt: 